In [12]:
import sys
from pathlib import Path
import numpy as np
import xarray as xr
import psyplot.project as psy
import iconarray as iconvis # import self-written modules from iconarray
from typing import List

In [13]:
class Point_coordinates:
    def __init__(self, shortname, name, color, marker, lat, lon, altitude, iconID, iconHSURF, iconHHL, iconHFL, height_dict, subsetID,station_type,provider ):
        self.short = shortname
        self.name = name
        self.color = color
        self.marker = marker
        self.lat = lat
        self.lon = lon
        self.alt = altitude
        self.iconID = iconID 
        self.iconHSURF = iconHSURF
        self.iconHHL = iconHHL
        self.iconHFL = iconHFL
        self.hdict = height_dict 
        self.subsetID = subsetID
        self.type = station_type
        self.provider = provider

In [14]:
# Start by loading dict created with Toni's Metadata from Stations.csv
loadpath= '/users/tlezuo/toni_codes/PLOTS/'
point_coordinates_dict = np.load(loadpath+'point_coordinates_dict.npy',allow_pickle='TRUE').item()

In [15]:
for k, v in point_coordinates_dict.items():
    point_coordinates_dict[k].station_type = 'toni'

In [16]:
# Append all extra locations I want to have
point_coordinates_dict['inn']=Point_coordinates('inn','Innsbruck', 'k','.',47.26,11.42,0,0,0,[],[],{},0,'ibox','acinn')
point_coordinates_dict['sal']=Point_coordinates('sal','Salzburg', 'k','.', 47.79,13.09,0,0,0,[],[],{},0,'ibox','acinn')
point_coordinates_dict['mun']=Point_coordinates('mun','Munich','k','.',48.14,11.57,0,0,0,[],[],{},0,'ibox','acinn')
point_coordinates_dict['bz']=Point_coordinates('bz','Bozen','k','.', 46.49,11.36,0,0,0,[],[],{},0,'ibox','acinn')
point_coordinates_dict['ver']=Point_coordinates('ver','Verona', 'k','.',45.43,10.99,0,0,0,[],[],{},0,'ibox','acinn')
point_coordinates_dict['ven']=Point_coordinates('ven','Venice','k','.',45.33,12.27,0,0,0,[],[],{},0,'ibox','acinn')
point_coordinates_dict['iun']=Point_coordinates('iun','stat Universität','gold',"$u$",47.2642889,11.3861614,0,0,0,[],[],{},0,'ibox','acinn')
point_coordinates_dict['ifl']=Point_coordinates('ifl','Innsbruck Airport','gold',"$f$",47.25846,11.3521825,579,808260,0,[],[],{},0,'ibox','acinn')
point_coordinates_dict['murs']=Point_coordinates('murs','Munich','gold',".",48.25,11.55,492,753969,0,[],[],{},0,'ibox','acinn')
point_coordinates_dict.update({'hoch':Point_coordinates('hoch','Hochhaueser','violet',"$h$",47.28755, 11.63122,1009,810547,0,[],[],{},0,'ibox','acinn')})
point_coordinates_dict['kols']= Point_coordinates('kols','Kolsass','darkviolet',"$k$",47.305, 11.622,547,810552,0,[],[],{},0,'ibox','acinn')
point_coordinates_dict['egg']=Point_coordinates('egg','Eggen','violet',"$e$",47.3165,11.6162,892,810583,0,[],[],{},0,'ibox','acinn')
point_coordinates_dict['weer']=Point_coordinates('weer','Weerberg','violet',"$w$",47.299,11.672,930,810560,0,[],[],{},0,'ibox','acinn')
point_coordinates_dict['terf']=Point_coordinates('terf','Terfens','violet',"$t$", 47.325538,11.65247,575,810571,0,[],[],{},0,'ibox','acinn')
point_coordinates_dict['arb']=Point_coordinates('arb','Arbeser','violet',"$a$",47.320654,11.746592,2015,811094,0,[],[],{},0,'ibox','acinn')

 ## ADD NEW LOCS HERE ###
# point_coordinates_dict['AAAAAAAAAAAAAAAAAAAAAAAA']=
# point_coordinates_dict['AAAAAAAAAAAAAAAAAAAAAAAA']=
# point_coordinates_dict['AAAAAAAAAAAAAAAAAAAAAAAA']=
# point_coordinates_dict['AAAAAAAAAAAAAAAAAAAAAAAA']=

In [17]:
## GET icon gridpoint id to include it in the lf.loc.iconID for new stations

# load cont file
nc_file_c = '/store/s83/tlezuo/RUN2_reference/lfff00000000c.nc'
data_c = psy.open_dataset(nc_file_c)

# get model grid information
HFL = (data_c['HHL'][1:].values + (data_c['HHL'][:-1].values-data_c['HHL'][1:].values)/2)
HHL = data_c['HHL'].values
HSURF = data_c['HSURF'].values
lats = np.rad2deg(data_c.clat.values[:])
lons = np.rad2deg(data_c.clon.values[:])

In [18]:
# ## THIERRY VERSION
# # 
# # # get dict information into lists
# pc_short_list = point_coordinates_dict.keys()
# pc_values: List[Point_coordinates] = point_coordinates_dict.values()
# lats_stat = [c.lat for c in pc_values]
# lons_stat = [c.lon for c in pc_values]

# # get index and save it in list
# pc_iconID_list = [iconvis.ind_from_latlon(lats,lons,lat,lon,verbose=True)for lat,lon in zip(lats_stat,lons_stat)]
# # put this list into pc_dict
# out_dict = {s.iconID: t for s, t in zip(pc_short_list, pc_iconID_list)}

# # get dict lists
# pc_short_list = point_coordinates_dict.keys()
# pc_values: List[Point_coordinates] = point_coordinates_dict.values()
# pc_iconID_list = [c.iconID for c in pc_values]

# # idee von Thierry um Zeitreihen ins point coordinate dict zu tun
# # temps = [] # TODO
# # out_dict = {s: t for s, t in zip(shorts, temps)}

In [19]:
pc_iconID_list = []
pc_short_list = ['LAN','IMS','HAI','ZUG','KUF','KIE','ROS','MUC','ALP','egg','terf','hoch','kols','weer','arb','RIN','UNI','ifl','murs']

for subsetID,key in enumerate(pc_short_list):
    loc = point_coordinates_dict[key]
    # save subsetID in dict
    point_coordinates_dict[key].subsetID = subsetID

    # get iconID
    ind = iconvis.ind_from_latlon(lats,lons,loc.lat,loc.lon,verbose=True)
    point_coordinates_dict[key].iconID = ind
    pc_iconID_list.append(ind)

    # get model altitudes
    HFL_loc = HFL[:,ind][::-1]
    point_coordinates_dict[key].iconHFL = HFL_loc
    HHL_loc = HHL[:,ind][::-1]
    point_coordinates_dict[key].iconHHL = HHL_loc
    HSURF_loc = HSURF[ind]
    point_coordinates_dict[key].iconHSURF = HSURF_loc

    # print('the coordinates for this ind are lat= '+str(np.rad2deg(data_c.clat[ind].values))+' lon= '+str(np.rad2deg(data_c.clon[ind].values)))
    print('the grid index for '+loc.name+' is '+str(ind))
    # print('the lowest level has a height of half level of '+str(np.round(HFL_loc[0])))
    # print('the lowest level has a height of full level of '+str(np.round(HHL_loc[0])))
    # print('the model topography is '+str(np.round(HSURF_loc)))


Closest ind: 476700
 Given lat: 47.140 vs found lat: 47.140
 Given lon: 10.564 vs found lon: 10.563
the grid index for Landeck is 476700
Closest ind: 475989
 Given lat: 47.237 vs found lat: 47.238
 Given lon: 10.742 vs found lon: 10.742
the grid index for Imst is 475989
Closest ind: 807401
 Given lat: 47.260 vs found lat: 47.260
 Given lon: 10.889 vs found lon: 10.892
the grid index for Haiming is 807401
Closest ind: 815236
 Given lat: 47.421 vs found lat: 47.416
 Given lon: 10.985 vs found lon: 10.988
the grid index for Zugspitze is 815236
Closest ind: 844147
 Given lat: 47.575 vs found lat: 47.572
 Given lon: 12.163 vs found lon: 12.164
the grid index for Kufstein is 844147
Closest ind: 844050
 Given lat: 47.619 vs found lat: 47.620
 Given lon: 12.166 vs found lon: 12.173
the grid index for Kiefersfelden-Gach is 844050
Closest ind: 817598
 Given lat: 47.875 vs found lat: 47.868
 Given lon: 12.128 vs found lon: 12.131
the grid index for Rosenheim is 817598
Closest ind: 816461
 Given l

KeyboardInterrupt: 

In [ ]:
## UPDATE height dictionaries ##
point_coordinates_dict['hoch'].hdict = {'T_2M':6.83,'VEL_10M':7.08,'DIR_10M':7.08, 'TKEs':1.50,'turb2':7.08,'rad':2.00, }
point_coordinates_dict['kols'].hdict = {'T_2M':8.68,#2,4,8.68,16.93
                                                                            'VEL_10M':8.68, #2,4,6,12
                                                                            'DIR_10M':8.68,
                                                                            'TKEs':4.00,
                                                                            'turb2':8.68,
                                                                            'turb3':16.93,
                                                                            'rad':2.00,
                                                                            }

In [ ]:
# save dict itself
np.save('pc_dict.npy',point_coordinates_dict)

In [ ]:
# get shortnames of dict - not needed anymore as list selected explicitly above
# pc_values: List[Point_coordinates] = point_coordinates_dict.values()
# pc_short_list = [c.short for c in pc_values]
 
# save pc_dict_lists
import pickle
with open ('pc_iconID_list','wb') as f:
    pickle.dump(pc_iconID_list,f)
with open ('pc_short_list','wb') as f:
    pickle.dump(pc_short_list,f)